# Adaptive Cards Toolkit: 03 - Data Visualization

This notebook demonstrates how to create cards with data visualizations like charts and tables. Please run 01_toolkit_common.ipynb first.

## Setup

First, we need to import the common utilities defined in the toolkit_common notebook.

In [ ]:
%run 01_toolkit_common.ipynb

# Ensure we have all the necessary imports
import sys
import os

# Add the parent directory to the path so we can import the toolkit
if not os.path.abspath('..') in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

# Import required modules directly
from src.adaptive_cards_toolkit.core.element_factory import ElementFactory
from src.adaptive_cards_toolkit.core.layout_helper import LayoutHelper
from src.adaptive_cards_toolkit.core.data_connector import DataConnector

## Create Sample Data

First, let's create some sample sales data for our visualization.

In [ ]:
# Create sample data - sales performance by region
regional_data = {
    "regions": ["North", "South", "East", "West", "Central"],
    "sales": [342000, 287000, 411000, 394000, 218000],
    "target": [350000, 275000, 400000, 425000, 225000],
    "reps": [12, 9, 15, 14, 8]
}

# Convert to pandas DataFrame
df = pd.DataFrame(regional_data)

# Calculate whether targets were met and percentage of target
df["target_met"] = df["sales"] >= df["target"]
df["percent_of_target"] = (df["sales"] / df["target"] * 100).round(1)
df["sales_per_rep"] = (df["sales"] / df["reps"]).round(0)

# Display the dataframe
df

## Create a Visualization

Now let's create a visualization of the sales data using seaborn and matplotlib.

In [ ]:
# Create a visualization
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))
bar_plot = sns.barplot(
    x="regions", 
    y="sales", 
    data=df,
    palette=["green" if met else "red" for met in df["target_met"]]
)

# Add target markers
for i, target in enumerate(df["target"]):
    plt.plot([i-0.4, i+0.4], [target, target], color="black", linestyle="--")

plt.title("Sales vs. Target by Region")
plt.ylabel("Sales ($)")
plt.tight_layout()

# Display the plot
plt.show()

# Convert to base64 for embedding in card
chart_img = fig_to_base64(plt.gcf())
plt.close()

## Create the Data Card

Now let's create a card to display this data and visualization.

In [ ]:
# Create a card to display this data
try:
    data_card = builder.create_basic_card(
        title="Regional Sales Performance",
        message="Q2 2024 sales performance by region compared to targets."
    )
    
    print("Created basic card!")
    print("Card type:", type(data_card))
    
    # Try to add our elements
    print("Preparing elements to add...")
    
    # Prepare all elements to add
    elements = []
    
    # Add chart image
    elements.append(ElementFactory.create_image(
        url=chart_img,
        alt_text="Bar chart showing sales vs target by region"
    ))
    
    # Generate table element manually since dataframe_to_table is not available
    try:
        print("Creating table from DataFrame manually...")
        # Convert DataFrame to a table using the available method in DataConnector
        headers = ["Region", "Sales ($)", "Target ($)", "% of Target", "Sales per Rep"]
        rows = []
        
        for _, row in df.iterrows():
            rows.append([
                row["regions"],
                f"${row['sales']:,.0f}",
                f"${row['target']:,.0f}",
                f"{row['percent_of_target']}%",
                f"${row['sales_per_rep']:,.0f}"
            ])
        
        table_elements = data_connector.create_table(
            headers=headers,
            rows=rows,
            has_header_row=True,
            alternate_row_style=True
        )
        
        # Add each table element to our elements list
        elements.extend(table_elements)
    except Exception as e:
        print(f"Error creating table: {e}")
    
    # Add heading for insights
    elements.append(ElementFactory.create_heading("Key Takeaways", level=2))
    
    # Generate insights
    best_region = df.loc[df["sales"].idxmax(), "regions"]
    worst_region = df.loc[df["sales"].idxmin(), "regions"]
    best_performer = df.loc[df["sales_per_rep"].idxmax(), "regions"]
    regions_over_target = df[df["target_met"] == True].shape[0]
    total_regions = df.shape[0]

    insights = [
        f"**{best_region}** had the highest total sales at ${df['sales'].max():,}.",
        f"**{worst_region}** had the lowest total sales at ${df['sales'].min():,}.",
        f"**{best_performer}** had the highest sales per representative at ${df['sales_per_rep'].max():,.0f}.",
        f"{regions_over_target} out of {total_regions} regions met or exceeded their sales targets."
    ]

    # Add each insight without using wrap parameter (it's already included in create_text implementation)
    for insight in insights:
        elements.append(ElementFactory.create_text(f"• {insight}"))
    
    # Now try different approaches to add the elements
    print(f"Adding {len(elements)} elements to card...")
    
    # Try different methods to add elements
    if hasattr(data_card, 'add_element'):
        print("Using add_element method...")
        for element in elements:
            data_card.add_element(element)
    elif hasattr(data_card, 'add_item'):
        print("Using add_item method...")
        for element in elements:
            data_card.add_item(element)
    elif hasattr(data_card, 'add'):
        print("Using add method...")
        for element in elements:
            data_card.add(element)
    elif hasattr(data_card, 'body') and hasattr(data_card.body, 'append'):
        print("Appending to body attribute...")
        for element in elements:
            data_card.body.append(element)
    elif hasattr(data_card, 'to_dict'):
        print("Modifying dictionary representation...")
        card_dict = data_card.to_dict()
        if 'body' in card_dict and isinstance(card_dict['body'], list):
            for element in elements:
                element_dict = element.to_dict() if hasattr(element, 'to_dict') else element
                card_dict['body'].append(element_dict)
    else:
        print("Could not find a way to add elements to the card.")
    
    # Try to add actions
    try:
        print("Adding actions...")
        actions = [
            {
                "type": "Action.Submit",
                "title": "Download Full Report",
                "data": {"action": "download_report"}
            },
            {
                "type": "Action.Submit",
                "title": "Schedule Review Meeting",
                "data": {"action": "schedule_meeting"}
            }
        ]
        
        if hasattr(data_card, 'add_action'):
            for action in actions:
                data_card.add_action(action)
        elif hasattr(data_card, 'actions') and hasattr(data_card.actions, 'append'):
            for action in actions:
                data_card.actions.append(action)
        elif hasattr(data_card, 'to_dict'):
            card_dict = data_card.to_dict()
            if 'actions' not in card_dict:
                card_dict['actions'] = []
            for action in actions:
                card_dict['actions'].append(action)
    except Exception as e:
        print(f"Error adding actions: {e}")
    
    # Display the final card
    display_card(data_card, "Data Visualization Card")
    
except Exception as e:
    print(f"Error creating data card: {e}")